# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

## Inicio do Código

In [6]:
# Bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Criar um Navegador
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

# Importar/Visualizar a base de dados
tabela = pd.read_excel('buscas.xlsx')
display(tabela)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,SmartWatch,Apple,150,250
1,Camisa Nike,Adidas,50,100


## Definição de Funções

In [7]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_nomes_obrigatorios, nome):
    tem_todas_palavras = True
    for palavra in lista_nomes_obrigatorios:
        if palavra not in nome:
            tem_todas_palavras = False
    return tem_todas_palavras


def busca_google_shopping(navegador, produto, termos_banidos, preco_maximo, preco_minimo):
    lista_ofertas = []
    #Tratamento dos Dados
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_nomes_obrigatorios = produto.split(' ')
    lista_termos_banidos = termos_banidos.split(' ')       
    #Entrar no Google
    navegador.get('https://www.google.com/')
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto, Keys.ENTER)
    #Entrar na Aba Shopping
    elementos = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break
    #Pegar as informações
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'i0X6df')
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()       
        #Analisar se não tem NENHUM termo BANIDO
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        #Analisar se tem TODOS os termos obrigatórios
        tem_todas_palavras = verificar_tem_todos_termos_produto(lista_nomes_obrigatorios, nome)        
        #Selecionar apenas os resultados que tem_termos_banidos = False E tem_todas_palavras = True
        if not tem_termos_banidos and tem_todas_palavras:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace('R$ ', '').replace('.', '').replace(',', '.')
            preco = float(preco)
        #Selecionar apenas entre os preços MAXIMO e MINIMO
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link= elemento_pai.get_attribute('href')
                lista_ofertas.append((nome,preco,link))
    return lista_ofertas


def busca_buscape(navegador, produto, termos_banidos, preco_maximo, preco_minimo):
    #Tratamento dos Dados
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_nomes_obrigatorios = produto.split(' ')
    lista_termos_banidos = termos_banidos.split(' ')
    #Buscar no Buscape
    navegador.get('https://www.buscape.com.br/')
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    #Pegar os resultados
    lista_ofertas = []
    while len(navegador.find_elements(By.CLASS_NAME, 'Select_Select__1S7HV')) <1:
        time.sleep(1)
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
        link = resultado.get_attribute('href')
        #Analisar se não tem NENHUM termo BANIDO
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        #Analisar se tem TODOS os termos obrigatórios
        tem_todas_palavras = verificar_tem_todos_termos_produto(lista_nomes_obrigatorios, nome)
        #Selecionar apenas entre os preços MAXIMO e MINIMO
        try:
            if not tem_termos_banidos and tem_todas_palavras: 
                preco = preco.replace('R$ ', '').replace('.', '').replace(',', '.')
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome,preco,link))
        except:
            continue
    return lista_ofertas

## Construindo as Tabelas

In [8]:
tabela_ofertas = pd.DataFrame()

for linha in tabela.index:
    #Pesquisar pelo produto
    produto = tabela.loc[linha, 'Nome']
    termos_banidos = tabela.loc[linha, 'Termos banidos']
    preco_maximo = tabela.loc[linha, 'Preço máximo']
    preco_minimo = tabela.loc[linha, 'Preço mínimo']
    
    resultados_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_maximo, preco_minimo)
    if resultados_google_shopping:
        tabela_google_shopping = pd.DataFrame(resultados_google_shopping, columns=['Produto','Preço','Link'])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_google_shopping])
        
    resultados_busca_buscape = busca_buscape(navegador, produto, termos_banidos, preco_maximo, preco_minimo)
    if resultados_busca_buscape:
        tabela_buscape = pd.DataFrame(resultados_busca_buscape, columns=['Produto','Preço','Link'])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_buscape])
        
display(tabela_ofertas)
    

,Produto,Preço,Link
0,smartwatch iwo8 44mm samsung preto,189.99,https://www.google.com/url?url=https://produto...
1,"smartwatch kabum! smart 550, preto, pulseira e...",179.00,https://www.google.com/url?url=https://www.kab...
2,smartwatch militar impermeável masculino com b...,232.66,https://www.google.com/url?url=https://produto...
3,smartwatch bluetooth sport à prova d'água lokm...,211.91,https://www.google.com/url?url=https://produto...
4,smartwatch triplamente resistente. generic,188.64,https://www.google.com/url?url=https://produto...
5,o mais novo logotipo do relógio smartwatch ult...,163.83,https://www.google.com/url?url=https://shopee....
6,relógio inteligente smartwatch w34 s com duas ...,168.20,https://www.google.com/url?url=https://www.mag...
7,de smartwatch militar anti-choque à prova d'ág...,190.51,https://www.google.com/url?url=https://produto...
8,smartwatch w26 lite preto troca pulseira faz l...,213.99,https://www.google.com/url?url=https://www.ame...
9,smartwatch - relógio inteligente - x8 pro max iwo,179.99,https://www.google.com/url?url=https://www.sho...


## Salvar no Excel

In [9]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

## Envio de Email

In [10]:
import win32com.client as win32

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('Outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'thiago100antonio@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f'''
    <p>Prezados, </p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada </p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.,</p>
    '''
    mail.Send()